In [1]:
!conda install -y cuda -c nvidia/label/cuda-11.4.0

Solving environment: done

# All requested packages already installed.



In [2]:
!conda install -y cuquantum -c conda-forge 

Solving environment: done

# All requested packages already installed.



In [3]:
!pip install cuquantum

In [4]:
!pip install torch torchvision

In [5]:
!pip install  PennyLane==0.27.0 PennyLane-Lightning==0.27.0 Pennylane-lightning-gpu==0.27.0

In [6]:
!pip install idx2numpy

In [8]:
!pip install medmnist

In [1]:
device = "cpu"
#device = "cuda"
import time,os,copy,torch,random_equivalent,torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from tqdm import tqdm
import pickle
# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(0)
np.random.seed(0)

# Plotting
import matplotlib.pyplot as plt


# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "16"

In [2]:
def get_data():
    # pip install medmnist
    import medmnist
    from medmnist import INFO
    data_flag = 'pneumoniamnist'

    info = INFO[data_flag]
    DataClass = getattr(medmnist, info['python_class'])
    pil_dataset_train = DataClass(split='train', download=True)
    pil_dataset_test = DataClass(split='test', download=True)
    return pil_dataset_train, pil_dataset_test

data_train, data_test = get_data()
arr = data_train.imgs/255.0
arr2 = data_train.labels
arr3 = []
arr4 = []
for i in range(len(arr)):
    if arr2[i] in [0,1]:
        arr3.append(arr[i])
        arr4.append(arr2[i])
# repeat the images for n_steps
n_steps = 10
# train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1))
# test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1))
train_images = np.asarray(arr3,dtype=np.float32) / 255.0
train_labels = np.asarray(arr4,dtype=np.int64)
train_labels = train_labels.squeeze()

#train_labels -= 6
display(np.shape(train_labels))

Using downloaded and verified file: /home/jovyan/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /home/jovyan/.medmnist/pneumoniamnist.npz


(4708,)

In [3]:

n_steps = 10
split = np.array_split(train_images,[int(len(train_images)*0.7)])
train_sequences = split[0]
test_sequences = split[1]
print(len(split[0]),len(split[1]))
split = np.array_split(train_labels,[int(len(train_labels)*0.7)])
train_labels = split[0]
test_labels = split[1]
print(len(split[0]),len(split[1]))
#datalength = 5000
datalength = 100

train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))[:datalength]
#train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))
test_sequences = np.tile(test_sequences[:, None], (1, n_steps, 1, 1))
train_labels = train_labels[:datalength]
test_labels = test_labels

print(len(test_labels),len(test_sequences),len(train_labels),len(train_sequences))

3295 1413
3295 1413
1413 1413 100 100


In [4]:
def train(input_model, train_x, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
 
    input_model.train()
    for j in range(30):
        train_acc = 0
        loss_acc = 0

        for i in tqdm(range(train_x.shape[0] // minibatch_size)):
            input_model.zero_grad()
 
            batch_in = train_x[i * minibatch_size : (i + 1) * minibatch_size]
            # flatten images
            batch_in = batch_in.reshape((-1,) + train_x.shape[1:-2] + (784,))
            batch_label = train_labels[i * minibatch_size : (i + 1) * minibatch_size]
            output = input_model(torch.tensor(batch_in))
 
            # compute sparse categorical cross entropy loss
            logp = torch.nn.functional.log_softmax(output, dim=-1)
            logpy = torch.gather(logp, 1, torch.tensor(batch_label).view(-1, 1))
            loss = -logpy.mean()
            loss_acc += loss 
 
            loss.backward()
            optimizer.step()
 
            train_acc += torch.mean(
                torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
            )
        train_acc /= i + 1
        #torch.save(model_hybrid,f=f"./saves_class10/{n_qubits}x{q_depth}_{j}_{train_acc.numpy()}.h5")  
        print("Train accuracy (%d): " % j, train_acc.numpy())
        loss_acc /=i + 1
        print("Loss (%d): " % j, loss_acc.detach().numpy())
        
    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))
 
        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )
 
    test_acc /= i + 1
 
    print("Test accuracy:", test_acc.numpy())
        
        

In [5]:
random_model = torch.nn.Sequential(
    torch.nn.Linear(784, 128),
    # set spiking_aware_training and a moderate dt
    random_equivalent.SpikingActivation(
    #Spike(
        torch.nn.ReLU(), dt=0.01, spiking_aware_training=True
    ),
    random_equivalent.TemporalAvgPool(),
    torch.nn.Linear(128, 10),
)



In [7]:
train(random_model, train_sequences, test_sequences)

100%|██████████| 3/3 [00:00<00:00,  5.10it/s]


Train accuracy (0):  0.26041666
Loss (0):  2.2664974


100%|██████████| 3/3 [00:00<00:00, 17.44it/s]


Train accuracy (1):  0.8125
Loss (1):  2.2145083


100%|██████████| 3/3 [00:00<00:00, 14.45it/s]


...

Train accuracy (27):  0.8125
Loss (27):  0.5289212


100%|██████████| 3/3 [00:00<00:00, 25.23it/s]


Train accuracy (28):  0.8125
Loss (28):  0.5233861


100%|██████████| 3/3 [00:00<00:00, 15.90it/s]


Train accuracy (29):  0.8125
Loss (29):  0.51884633
Test accuracy: 0.75923294


In [14]:
!mkdir -p HybridQRNN/MNIST

In [8]:
torch.save(random_model,"./HybridQRNN/MNIST/random_mnist_final.h5")

In [9]:
random_model = torch.load("./HybridQRNN/MNIST/random_mnist_final.h5")

In [10]:
n_qubits = 6            # Number of qubits
q_depth = 2
q_delta = 0.01 
nqubits=n_qubits
tensor_length = n_qubits*(n_qubits-1)*q_depth+n_qubits

In [11]:
def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RZ_layer(w):
    for idx, element in enumerate(w):
        qml.RZ(element, wires=idx)

def entangling_layer(nqubits,weights):
    p = nqubits
    weights_ = (weight for weight in weights)
    for i in range(1,nqubits):
        for j in range(i):
            qml.CNOT(wires=[j,i])
            param = next(weights_)
            # vqc_params.append(param)
            qml.RZ(param, wires=i)
            p+=1
            param = next(weights_)
            # vqc_params.append(param)
            qml.RX(param, wires=i)
            p+=1
            qml.CNOT(wires=[j,i])
            
def entangling_layer1(nqubits,weights):
    n=nqubits
    weights_ = (weight for weight in weights)
    for i in range(1,n):
        j = 0
        r = []
        while j <= n-i-1:
            if j not in r:
                qml.CNOT([j,j+i])
                r += [j+1]
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([j,j+i])
            j += 1
        for j in r:
            if j+i <= n-1:
                qml.CNOT([j,j+i])
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([j,j+i])

In [4]:
'''#draw the modified layer
wiress = [0,1,2,3,5,6]
np.random.seed(0)
weightss= 0.01 * np.random.randn(len(wiress)*(len(wiress)-1), requires_grad=True)

dev = qml.device("default.qubit", wires=wiress)
@qml.qnode(dev)

def entangling_layer1(wires,weights):
    n=len(wires)
    weights_ = (weight for weight in weights)
    for i in range(1,n):
        j = 0
        r = []
        while j <= n-i-1:
            if j not in r:
                qml.CNOT([wires[j],wires[j+i]])
                r += [j+1]
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([wires[j],wires[j+i]])
            j += 1
        for j in r:
            if j+i <= n-1:
                qml.CNOT([wires[j],wires[j+i]])
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([wires[j],wires[j+i]])
    return qml.probs(wires[0])
            
print(qml.draw(entangling_layer1)(wiress,weightss))

0: ─╭●─────────RZ(0.02)───RX(0.00)─╭●─────────────────────────────╭●──RZ(0.00)──RX(0.01)─╭●───
1: ─╰X─────────────────────────────╰X─╭●──RZ(0.01)───RX(-0.00)─╭●─│──────────────────────│────
2: ─╭●─────────RZ(0.01)───RX(0.02)─╭●─╰X───────────────────────╰X─╰X─────────────────────╰X─╭●
3: ─╰X─────────────────────────────╰X─╭●──RZ(-0.00)──RX(0.00)──╭●───────────────────────────│─
5: ─╭●────────╭●──────────────────────╰X───────────────────────╰X───────────────────────────╰X
6: ─╰X────────╰X──────────────────────────────────────────────────────────────────────────────
4: ──RZ(0.02)──RX(-0.01)──────────────────────────────────────────────────────────────────────

──────────────────────────────────────────────────────────────────────────────╭●──RZ(0.00)
─────────────────────────╭●──RZ(0.00)──RX(0.00)─╭●────────────────────────────│───────────
───RZ(0.01)──RX(0.00)─╭●─│──────────────────────│─────────────────────────────│───────────
──────────────────────│──╰X─────────────────────╰X─╭●──RZ(0.0

In [12]:
dev = qml.device("lightning.gpu", wires=n_qubits)
@qml.qnode(dev, interface="torch")

def quantum_net(q_input_features, q_weights_flat):
    q_weights = q_weights_flat
    H_layer(n_qubits)
    RZ_layer(q_input_features)
    entangling_layer1(nqubits,q_weights)
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)
    
class DressedQuantumNet(nn.Module):

    def __init__(self):

        super().__init__()
        self.pre_net = nn.Linear(128, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(tensor_length))
        self.post_net = nn.Linear(n_qubits, 5)
        self.ppost_net = nn.Linear(5, 2)

    def forward(self, input_features):

        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))
            
        return self.ppost_net(self.post_net(q_out))

In [13]:
model_hybrid = copy.deepcopy(torch.nn.Sequential(*(list(random_model.children())[:-1])))
for param in model_hybrid.parameters():
    param.requires_grad = False

model_hybrid.fc = DressedQuantumNet()

model_hybrid = model_hybrid.to(device)

In [14]:
def train(input_model, train_x, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
 
    input_model.train()
    for j in range(100):
        train_acc = 0
        loss_acc = 0
        
        for i in tqdm(range(train_x.shape[0] // minibatch_size)):
            input_model.zero_grad()
 
            batch_in = train_x[i * minibatch_size : (i + 1) * minibatch_size]

            batch_in = batch_in.reshape((-1,) + train_x.shape[1:-2] + (784,))
            batch_label = train_labels[i * minibatch_size : (i + 1) * minibatch_size]
            output = input_model(torch.tensor(batch_in))
 
            logp = torch.nn.functional.log_softmax(output, dim=-1)
            logpy = torch.gather(logp, 1, torch.tensor(batch_label).view(-1, 1))
            loss = -logpy.mean()
            loss_acc += loss 
 
            loss.backward()
            optimizer.step()
 
            train_acc += torch.mean(
                torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
            )
        train_acc /= i + 1  
        print("Train accuracy (%d): " % j, train_acc.numpy())
        metric_train=train_acc.numpy()

        loss_acc /=i + 1
        print("Loss (%d): " % j, loss_acc.detach().numpy())
        torch.save(model_hybrid,f="./model_saves/4_class/6x2_zz_{0}_{1}_{2}.h5".format(j+21,train_acc.numpy(),loss_acc.detach().numpy()))
        metric_loss= loss_acc.detach().numpy()
        #metric.append([metric_train,metric_loss])
        #np.save("metrics.npy",np.array(metric))
        # compute test accuracy
        
    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))
         
        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )
 
    test_acc /= i + 1
 
    print("Test accuracy:", test_acc.numpy())

In [15]:
def test(input_model, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())

In [16]:
!mkdir -p model_saves/4_class

In [17]:
train(model_hybrid, train_sequences, test_sequences)

100%|██████████| 3/3 [00:21<00:00,  7.32s/it]


Train accuracy (0):  0.1875
Loss (0):  0.800572


100%|██████████| 3/3 [00:21<00:00,  7.17s/it]


Train accuracy (1):  0.1875
Loss (1):  0.7934551


100%|██████████| 3/3 [00:21<00:00,  7.26s/it]


...

Train accuracy (88):  0.8125
Loss (88):  0.48198986


100%|██████████| 3/3 [00:21<00:00,  7.29s/it]


Train accuracy (89):  0.8125
Loss (89):  0.48198023


 67%|██████▋   | 2/3 [00:20<00:10, 10.32s/it]


KeyboardInterrupt: 

In [55]:
print(qml.draw(quantum_net(q_input_features, q_weights_flat)))

NameError: name 'q_input_features' is not defined

In [ ]:
def test(input_model,test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
    arr = np.array([])
    input_model.eval()
    test_acc = 0
    
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))
#         print("Batch Label:",batch_label)
        arr=np.append(arr,torch.argmax(output, dim=1).numpy())
        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )
    
    test_acc /= i + 1
    print(f"Test accuracy:", test_acc.numpy())
    return test_acc.numpy()


In [ ]:
test_Acc_snp=[]
# test model with test dataset
for i in range(1,10):
    snp_noise_level=0.01*i

    test_image=(np.load(f"./saved_data_PneumoniaMNIST/snp_%.2f_42.npz"%(snp_noise_level))['arr_0'])
    test_X_snp=np.array([x[0] for x in test_image])
    test_label=np.load("./saved_data_PneumoniaMNIST/test_labels.npz")['arr_0']
    test_acc=test(hybrid_model,test_X_snp)
    test_Acc_snp.append(test_acc)

In [ ]:
!pip freeze > requirements.txt